In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w3p1/train.csv
/kaggle/input/2022-ai-w3p1/test.csv
/kaggle/input/2022-ai-w3p1/submit_sample.csv


In [2]:
import torch
import torch.optim as optim

import random

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
train = pd.read_csv('/kaggle/input/2022-ai-w3p1/train.csv')
test = pd.read_csv('/kaggle/input/2022-ai-w3p1/test.csv')
submission = pd.read_csv('/kaggle/input/2022-ai-w3p1/submit_sample.csv')

In [4]:
train.columns

Index(['year', 'avgTemp', 'minTemp', 'maxTemp', 'rainFall', 'avgPrice'], dtype='object')

In [5]:
x_train = np.array(train[['avgTemp', 'minTemp', 'maxTemp', 'rainFall']])
y_train = np.array(train['avgPrice'])
y_train = y_train.reshape(-1,1)
x_test = np.array(test[['avgTemp', 'minTemp', 'maxTemp', 'rainFall']])

In [6]:
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)
x_test = torch.FloatTensor(x_test)

In [7]:
print(x_train)
print(y_train)
print(x_test)

tensor([[ -4.9000, -11.0000,   0.9000,   0.0000],
        [ -3.1000,  -5.5000,   5.5000,   0.8000],
        [ -2.9000,  -6.9000,   1.4000,   0.0000],
        ...,
        [ -1.3000,  -4.5000,   2.1000,   0.2000],
        [ -1.4000,  -7.0000,   4.6000,   0.0000],
        [  1.7000,  -2.9000,   7.1000,   0.0000]])
tensor([[2123.],
        [2123.],
        [2123.],
        ...,
        [4224.],
        [4219.],
        [4219.]])
tensor([[ 4.0000, -1.4000,  9.5000,  0.1000],
        [-1.7000, -5.4000,  1.4000,  3.3000],
        [ 1.4000, -2.0000,  5.0000,  0.0000],
        [ 3.6000, -2.5000, 11.1000,  0.1000],
        [ 5.0000, -1.6000, 11.3000,  0.0000],
        [ 7.3000,  0.7000, 13.6000,  0.7000],
        [12.3000,  7.2000, 17.7000,  0.0000],
        [19.8000, 11.9000, 27.7000,  0.0000],
        [17.5000, 12.3000, 24.5000,  2.9000],
        [20.6000, 11.8000, 28.3000,  0.0000],
        [18.4000, 15.9000, 21.2000,  5.5000],
        [24.7000, 18.6000, 31.5000,  0.0000],
        [26.1000, 

In [8]:
W = torch.zeros((4,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=3)

nb_epochs = 50000
                      
for epoch in range(nb_epochs+1):
    hypothesis = x_train.matmul(W) + b
    cost = torch.sqrt(torch.mean((hypothesis - y_train) ** 2))
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
            epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
        ))

Epoch    0/50000 hypothesis: tensor([0., 0., 0.,  ..., 0., 0., 0.]) Cost: 3466.026611
Epoch  100/50000 hypothesis: tensor([2536.1782, 2576.7063, 1821.7051,  ..., 1512.1125, 2633.4438,
        2413.6895]) Cost: 1474.443481
Epoch  200/50000 hypothesis: tensor([2569.9277, 2633.3689, 1855.7998,  ..., 1543.8391, 2664.4563,
        2441.8550]) Cost: 1470.786011
Epoch  300/50000 hypothesis: tensor([2571.2358, 2663.0288, 1867.8232,  ..., 1558.9930, 2668.0928,
        2449.9639]) Cost: 1467.301636
Epoch  400/50000 hypothesis: tensor([2572.3206, 2691.7163, 1879.5417,  ..., 1573.8590, 2671.4741,
        2457.7998]) Cost: 1463.903198
Epoch  500/50000 hypothesis: tensor([2573.4307, 2719.6465, 1891.1246,  ..., 1588.5636, 2674.8110,
        2465.5176]) Cost: 1460.588745
Epoch  600/50000 hypothesis: tensor([2574.5667, 2746.8337, 1902.5741,  ..., 1603.1089, 2678.1047,
        2473.1191]) Cost: 1457.356567
Epoch  700/50000 hypothesis: tensor([2575.7278, 2773.2913, 1913.8906,  ..., 1617.4952, 2681.3562,


In [9]:
test_re = x_test.matmul(W) + b
test_re

tensor([[2996.9236],
        [2773.8406],
        [2866.2437],
        [3114.4414],
        [3039.2290],
        [3086.0867],
        [3157.6470],
        [3402.6997],
        [3381.9888],
        [3377.4751],
        [3181.6699],
        [3469.0850],
        [3394.1963],
        [3413.6877],
        [3354.5059],
        [3434.2998],
        [3390.4719],
        [3420.0051],
        [3393.3638],
        [3246.9092],
        [3006.3479],
        [3021.9429],
        [2896.7268],
        [2835.8154]], grad_fn=<AddBackward0>)

In [10]:
submission['Expected'] = test_re.detach().numpy()
submission=submission.astype(np.int32)
submission.to_csv('submission.csv',mode='w',header=True,index=False)